In [1]:
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
folder_path = "D:\\ms\\January 2024\\Thesis\\Boiler-Bidding-model\\Datasets\\"

linheat_df = pd.read_csv(folder_path +'boiler-12025041008_15_13_LHP_2_PHLIT_export.csv')
linheat_df['UTC'] = pd.to_datetime(linheat_df['UTC'])

In [4]:
linheat_df.columns

Index(['UTC', 'Heat load forecast [MW]', 'L11 Heat storage energy [MWh]',
       'L11 EK plan [MW]', 'L13 HR plan [MW]', 'L11 VP plan [MW]',
       'L02 VP plan [MW]', 'L31 VP plan [MW]', 'L11 Sonfor plan [MW]',
       'L11 K5 plan [MW]', 'L11 K8 plan [MW]', 'D41 VP MDC plan [MW]',
       'Total heat production plan [MW]', 'Spotprice Prediction [DKK/MWh]',
       'Spotprice [DKK/MWh]'],
      dtype='object')

In [14]:
temp = linheat_df[['UTC','Heat load forecast [MW]','L11 EK plan [MW]']].iloc[0:96]

In [3]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01)

# Add traces
fig.add_trace(go.Scatter(x=linheat_df['UTC'], y=linheat_df['Heat load forecast [MW]'], mode='lines', name='Heat load forecast [MW]'), row=1, col=1)
fig.add_trace(go.Scatter(x=linheat_df['UTC'], y=np.ceil(linheat_df['Heat load forecast [MW]']), mode='lines', name='Heat load forecast [MW] rounded'), row=1, col=1)
fig.add_trace(go.Scatter(x=linheat_df['UTC'], y=np.ceil(linheat_df['L11 EK plan [MW]']), mode='lines', name='L11 EK plan [MW]'), row=1, col=1)

fig.add_trace(go.Scatter(x=linheat_df['UTC'], y=linheat_df['Heat load forecast [MW]'] * 0.25, mode='lines', name='Heat Demand Supplied [MWh]'), row=2, col=1)

# Update layout with size and borders
fig.update_layout(title='LinHeat schedule',xaxis_title='Time', yaxis1_title='MW', yaxis2_title='MWh', legend_title='Legend',template='plotly_white', margin=dict(t=60, b=60, l=60, r=60),  showlegend=True)

for i in range(1, 3):
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', row=i, col=1)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', row=i, col=1)

fig.show()

In [ ]:
from gurobipy import GRB
model = gp.Model('RTOptimisation')

horizon = 24 * 4 # 15 min blocks over a day
T = list(range(horizon))
seconds_MTU = list(range(4, 15 * 60 +1, 4)) # 4s intervals in a 15 minute step
deltaT = 4/3600  #4s interval in a hour

# demand parameters - These paramters are for each MTU
P_scheduled = 
demand = 

# boiler parameters
P_max = 12 #MW
Emax = 60 #MWh
eff_SOE = 0.99 #Mwh
eff_P2H = 0.97
Einit = 30 #MWh

# price parameters
lambda_cl_UP = price_data[:, :, columns['aFRR_UP_Price_prediction']] # These parameters are for every 4s interval of each MTU
lambda_cl_DOWN = -price_data[:, :, columns['aFRR_DOWN_Price_prediction']]
spotprice = price_data[:, 0, columns['SpotPriceEUR']]

# variables
P_up = model.addVars(T, lb = 0, vtype = GRB.INTEGER, name = 'P_up')
P_down = model.addVars(T, lb = 0, vtype = GRB.INTEGER, name = 'P_down')

# Auxillary Variables
P_boiler = model.addVars(T, lb = 0, ub = 12, vtype = GRB.INTEGER, name = 'P_boiler')
SoE = model.addVars(T, len(seconds_MTU), lb = 0, ub = 1, vtype = GRB.CONTINUOUS, name = 'SoE')
E_up = model.addVars(T, len(seconds_MTU), lb = 0, ub = 3, vtype = GRB.CONTINUOUS, name = 'E_up') # Rows - MTU Columns - 4s intervals
E_down = model.addVars(T, len(seconds_MTU), lb = 0, ub = 3, vtype = GRB.CONTINUOUS, name = 'E_down')


# Constraints
for t in T:
    model.addConstr(P_up[t] <= P_boiler[t], name = f'UP_Capacity_available_at_{t}')
    model.addConstr(P_down[t] <= P_max - P_boiler[t], name = f'Down_Capacity_available_at_{t}')

for t in T:
    for k in range(len(seconds_MTU)):
        model.addConstr(E_up[t,k] == eff_P2H * P_up[t] * deltaT, name = f'E_UP_a_{t}MTU_{seconds_MTU[k]}s')
        model.addConstr(E_down[t,k] == eff_P2H * P_down[t] * deltaT, name = f'E_UP_a_{t}MTU_{seconds_MTU[k]}s')

# Constraint for the System state after activation
model.addConstr(SoE[0,0] == eff_SOE * Einit/Emax + (E_down[0,0] - E_up[0,0])/Emax  - (deltaT * demand[0])/Emax, name = f'System_state_after_{0}MTU_{seconds_MTU[0]}s')

for k in range(len(seconds_MTU))[1:]:
    model.addConstr(SoE[0,k] == eff_SOE * SoE[0,k-1] + (E_down[0,k] - E_up[0,k])/Emax  - (deltaT * demand[0])/Emax, name = f'System_state_after_{0}MTU_{seconds_MTU[k]}s')

for t in T[1:]:
    model.addConstr(SoE[t,0] == eff_SOE * SoE[t-1, len(seconds_MTU)-1] + (E_down[t,0] - E_up[t,0])/Emax  - (deltaT * demand[t])/Emax, name = f'System_state_after_{t}MTU_{seconds_MTU[0]}s')

    for k in range(len(seconds_MTU)):
        model.addConstr(SoE[t,k] == eff_SOE * SoE[t,k-1] + (E_down[t,k] - E_up[t,k])/Emax  - (deltaT * demand[t])/Emax, name = f'System_state_after_{t}MTU_{seconds_MTU[k]}s')

# objective function

Z = gp.quicksum(
        gp.quicksum(lambda_cl_UP[t, k] * E_up[t, k] for k in range(len(seconds_MTU))) + 
        gp.quicksum(-lambda_cl_DOWN[t,k] * E_down[t,k] for k in range(len(seconds_MTU)))
        for t in T)

model.setObjective(Z, GRB.MAXIMIZE)
model.optimize()